In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bayuabdurrosyidyeye","key":"20598f24bbb6aa555f3c275afbfc6573"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d antonkozyriev/game-recommendations-on-steam
!kaggle datasets download -d nikdavis/steam-store-games

100% 570M/573M [00:06<00:00, 152MB/s]
100% 573M/573M [00:06<00:00, 93.2MB/s]
 71% 25.0M/35.2M [00:00<00:00, 110MB/s] 
100% 35.2M/35.2M [00:00<00:00, 125MB/s]


In [ ]:
!unzip game-recommendations-on-steam.zip
!unzip steam-store-games.zip

Archive:  game-recommendations-on-steam.zip
  inflating: games.csv               
  inflating: games_metadata.json     
  inflating: recommendations.csv     
  inflating: users.csv               
Archive:  steam-store-games.zip
  inflating: steam.csv               
  inflating: steam_description_data.csv  
  inflating: steam_media_data.csv    
  inflating: steam_requirements_data.csv  
  inflating: steam_support_info.csv  
  inflating: steamspy_tag_data.csv   


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
games = pd.read_csv('/content/steam.csv')
recommendations = pd.read_csv('/content/recommendations.csv')

In [ ]:
print("tabel data games\nukuran data: ", games.shape)
display(games.head())
print("tabel data recommendations\nukuran data: ", recommendations.shape)
display(recommendations.head())

tabel data games
ukuran data:  (27075, 18)


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


tabel data recommendations
ukuran data:  (35271521, 8)


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,47199,0
1,304390,4,0,2017-02-17,False,11.5,2376,1
2,1085660,2,0,2019-11-17,True,336.5,230757,2
3,703080,0,0,2022-09-23,True,27.4,235736,3
4,526870,0,0,2021-01-10,True,7.9,21721,4


In [ ]:
games.describe()

,appid,english,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price
count,2.707500e+04,27075.000000,27075.000000,27075.000000,2.707500e+04,27075.000000,27075.000000,27075.00000,27075.000000
mean,5.962035e+05,0.981127,0.354903,45.248864,1.000559e+03,211.027147,149.804949,146.05603,6.078193
std,2.508942e+05,0.136081,2.406044,352.670281,1.898872e+04,4284.938531,1827.038141,2353.88008,7.874922
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000
25%,4.012300e+05,1.000000,0.000000,0.000000,6.000000e+00,2.000000,0.000000,0.00000,1.690000
50%,5.990700e+05,1.000000,0.000000,7.000000,2.400000e+01,9.000000,0.000000,0.00000,3.990000
75%,7.987600e+05,1.000000,0.000000,23.000000,1.260000e+02,42.000000,0.000000,0.00000,7.190000
max,1.069460e+06,1.000000,18.000000,9821.000000,2.644404e+06,487076.000000,190625.000000,190625.00000,421.990000


In [ ]:
recommendations.describe()

,app_id,helpful,funny,hours,user_id,review_id
count,3.527152e+07,3.527152e+07,3.527152e+07,3.527152e+07,3.527152e+07,3.527152e+07
mean,6.054331e+05,3.387822e+00,1.124787e+00,1.053939e+02,6.858198e+06,1.763576e+07
std,4.710976e+05,4.950573e+01,3.035456e+01,1.798430e+02,3.705082e+06,1.018201e+07
min,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.589700e+05,0.000000e+00,0.000000e+00,8.400000e+00,3.918894e+06,8.817880e+06
50%,4.381000e+05,0.000000e+00,0.000000e+00,2.980000e+01,6.947822e+06,1.763576e+07
75%,9.453600e+05,0.000000e+00,0.000000e+00,1.071000e+02,1.011698e+07,2.645364e+07
max,2.253290e+06,3.621200e+04,2.808800e+04,9.999000e+02,1.317986e+07,3.527152e+07


In [ ]:
display(games.categories.unique())
print('Jumlah total kategori game: ', len(games.categories.unique()))
display(games.genres.unique())
print('Jumlah total genre game: ', len(games.genres.unique()))
display(games.steamspy_tags.unique())
print('Jumlah total tag game: ', len(games.steamspy_tags.unique()))

array(['Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled',
       'Multi-player;Valve Anti-Cheat enabled',
       'Single-player;Multi-player;Valve Anti-Cheat enabled', ...,
       'Online Multi-Player;Steam Achievements;Full controller support;In-App Purchases;Steam Cloud',
       'Multi-player;Local Multi-Player;Co-op;Local Co-op;Shared/Split Screen',
       'Multi-player;Online Multi-Player;Cross-Platform Multiplayer;Stats'],
      dtype=object)

Jumlah total kategori game:  3333


array(['Action', 'Action;Free to Play', 'Action;Free to Play;Strategy',
       ...,
       'Action;Adventure;Indie;Massively Multiplayer;RPG;Strategy;Early Access',
       'Action;Adventure;Casual;Free to Play;Indie;RPG;Simulation;Sports;Strategy',
       'Casual;Free to Play;Massively Multiplayer;RPG;Early Access'],
      dtype=object)

Jumlah total genre game:  1552


array(['Action;FPS;Multiplayer', 'FPS;World War II;Multiplayer',
       'FPS;Action;Sci-fi', ..., 'Casual;Adventure;Arcade',
       'Free to Play;Visual Novel',
       'Early Access;Adventure;Sexual Content'], dtype=object)

Jumlah total tag game:  6423


## Content based filtering

In [ ]:
games['tag'] = games.apply(lambda row: row['categories'] + '; ' + row['genres'] + '; ' + row['steamspy_tags'], axis=1)
games.drop(columns=['categories', 'genres', 'steamspy_tags'], inplace=True)
games.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,tag
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,0,124534,3339,17612,317,10000000-20000000,7.19,Multi-player;Online Multi-Player;Local Multi-P...
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,0,3318,633,277,62,5000000-10000000,3.99,Multi-player;Online Multi-Player;Local Multi-P...
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,0,3416,398,187,34,5000000-10000000,3.99,Multi-player;Valve Anti-Cheat enabled; Action;...
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,0,1273,267,258,184,5000000-10000000,3.99,Multi-player;Online Multi-Player;Local Multi-P...
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,0,5250,288,624,415,5000000-10000000,3.99,Single-player;Multi-player;Valve Anti-Cheat en...


In [ ]:
tf = TfidfVectorizer()
tf.fit(games['tag'])
tf.get_feature_names_out()

array(['1980s', '1990', '2d', '360', '3d', '40k', '4x', '5d', '6dof',
       'abstract', 'access', 'accounting', 'achievements', 'action',
       'adventure', 'agriculture', 'aliens', 'alternate', 'america',
       'and', 'animation', 'anime', 'anti', 'apocalyptic', 'app',
       'arcade', 'arena', 'arts', 'assassin', 'atmospheric', 'attack',
       'audio', 'available', 'awkward', 'base', 'baseball', 'based',
       'basketball', 'batman', 'battle', 'beat', 'beautiful', 'benchmark',
       'bikes', 'blood', 'bmx', 'board', 'book', 'bowling', 'builder',
       'building', 'bullet', 'capitalism', 'captions', 'card', 'cards',
       'cartoon', 'cartoony', 'casual', 'cats', 'character', 'cheat',
       'chess', 'choices', 'choose', 'cinematic', 'city', 'class',
       'classic', 'click', 'clicker', 'cloud', 'co', 'cold',
       'collectibles', 'colorful', 'combat', 'comedy', 'comic',
       'commentary', 'competitive', 'content', 'control', 'controller',
       'controls', 'conversation',

In [ ]:
tfidf_matrix = tf.fit_transform(games['tag'])
tfidf_matrix.shape

(27075, 420)

In [ ]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=games.name
).sample(22, axis=1).sample(10, axis=0)

,city,360,friendly,otome,great,illuminati,golf,rogue,batman,puzzle,...,soundtrack,trains,diplomacy,parody,pvp,paced,click,lego,kickstarter,games
name,,,,,,,,,,,,,,,,,,,,,
WAyE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ultimate Select Hero / 究极勇者的选择传说,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Thunder Chase,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cubway,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brothers - A Tale of Two Sons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Undead Souls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lichtspeer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Men of War: Condemned Heroes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jewel Tree,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=games.name, columns=games.name)
print('Shape:', cosine_sim_df.shape)
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (27075, 27075)


name,Perception,Bloodstained: Curse of the Moon,Airtone,Miracle Circus,Team Fortress Classic
name,,,,,
March of Empires,0.030025,0.043638,0.043026,0.264022,0.141877
Timber Tennis: Versus,0.179185,0.311625,0.134280,0.252766,0.325030
KRUNCH,0.245757,0.396342,0.267667,0.105556,0.066141
V.T.,0.357429,0.268783,0.397441,0.399423,0.221515
Demonicon,0.370460,0.454498,0.513280,0.569047,0.083873
The Cube Hotel（Ning's Wing 2）,0.315129,0.022679,0.324351,0.745638,0.022448
GM Forge - Virtual Tabletop,0.100291,0.111785,0.217996,0.363514,0.387647
From Darkness,0.108664,0.021293,0.036256,0.254970,0.021077
Forgotten Adventure,0.328082,0.023611,0.337683,0.511335,0.023370


In [ ]:
def content_recommender(game, similarity_data=cosine_sim_df, items=games[['name', 'tag']], k=5):

    index = similarity_data.loc[:,game].to_numpy().argpartition(
        range(-1, -k, -1))

    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    closest = closest.drop(game, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
games[games.name.eq("Assassin's Creed® III Remastered")]

,appid,name,release_date,english,developer,publisher,platforms,required_age,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,tag
23892,911400,Assassin's Creed® III Remastered,2019-03-29,1,Ubisoft Entertainment,Ubisoft Entertainment\t\t\t\t,windows,0,0,475,983,80,80,50000-100000,33.99,Single-player;Partial Controller Support; Acti...


In [ ]:
content_recommender("Assassin's Creed® III Remastered")

,name,tag
0,Assassin's Creed Freedom Cry,Single-player; Action;Adventure; Action;Advent...
1,Assassin’s Creed® Rogue,Single-player;Partial Controller Support; Acti...
2,Assassin’s Creed® Chronicles: China,Single-player;Steam Trading Cards;Partial Cont...
3,Assassin's Creed® Syndicate,Single-player;In-App Purchases;Partial Control...
4,Hitman: Contracts,Single-player; Action; Stealth;Action;Assassin


## Collaborative Filtering

In [ ]:
games.rename(columns={'appid':'app_id'}, inplace=True)

In [ ]:
recommend_games = pd.merge(recommendations, games[['app_id', 'name', 'tag']], on='app_id', how='left')
recommend_games.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id,name,tag
0,975370,0,0,2022-12-12,True,36.3,47199,0,NaN,NaN
1,304390,4,0,2017-02-17,False,11.5,2376,1,FOR HONOR™,Single-player;Online Multi-Player;Online Co-op...
2,1085660,2,0,2019-11-17,True,336.5,230757,2,NaN,NaN
3,703080,0,0,2022-09-23,True,27.4,235736,3,NaN,NaN
4,526870,0,0,2021-01-10,True,7.9,21721,4,NaN,NaN


In [ ]:
recommend_games.isna().sum()

app_id                   0
helpful                  0
funny                    0
date                     0
is_recommended           0
hours                    0
user_id                  0
review_id                0
name              11187145
tag               11187145
dtype: int64

In [ ]:
recommend_games.dropna(inplace=True)
recommend_games.reset_index(drop=True, inplace=True)
print(recommend_games.shape)
recommend_games.isna().sum()

(24084376, 10)


app_id            0
helpful           0
funny             0
date              0
is_recommended    0
hours             0
user_id           0
review_id         0
name              0
tag               0
dtype: int64

In [ ]:
user_ids = recommend_games['user_id'].astype('category').cat.codes
game_ids = recommend_games['app_id'].astype('category').cat.codes

In [ ]:
unique_user_ids = recommend_games['user_id'].astype('category').cat.categories
unique_game_ids = recommend_games['app_id'].astype('category').cat.categories

In [ ]:
user_game_matrix = coo_matrix((recommend_games['is_recommended'].apply(lambda x: int(x)), (user_ids, game_ids)))

In [ ]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_game_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

## Test

In [ ]:
def collaborative_based_filtering(id_user):

  user_query = np.where(unique_user_ids==id_user)[0][0]
  distances, indices = model_knn.kneighbors(user_game_matrix.getrow(user_query), n_neighbors=32)

  recommender = []
  for i in range(1, len(distances.flatten())):
    recommender.append(np.array(recommend_games.loc[recommend_games.user_id==unique_user_ids[indices.flatten()[i]]].name))

  max_string_length = max(len(s) for arr in recommender for s in arr)
  total_length = sum(arr.shape[0] for arr in recommender)
  x = np.empty(total_length, dtype=f'U{max_string_length}')

  start_idx = 0
  for arr in recommender:
    end_idx = start_idx + arr.shape[0]
    x[start_idx:end_idx] = arr
    start_idx = end_idx

  user_game =  set(np.array(recommend_games.loc[recommend_games.user_id==unique_user_ids[user_query]].name))
  rec = list(set(x) - user_game)

  print('\nPengguna lain juga memainkan:')
  if(len(rec) > 5):
    for i in range(5):
      print(i+1, '. ', rec[i], ': ', games[games.name == rec[i]].tag.values)
  else:
    for i in range(len(rec)):
      print(i+1, '. ', rec[i], ': ', games[games.name == rec[i]].tag.values)

In [ ]:
def content_based_filtering(id):
  user_favorite_games = recommend_games[recommend_games.user_id == id].sort_values('hours', ascending=False)
  print('Daftar game favorit pengguna id: ', id)
  for i in range(5):
    print(i+1, '. ', user_favorite_games.name.iloc[i], ': ', [user_favorite_games.tag.iloc[i]])
  print('\nRekomendasi game untuk pengguna id: ', id)
  for i in range(5):
    x = content_recommender(user_favorite_games.name.iloc[i]).head(1)
    if (x.name[0] in user_favorite_games):
      i = i - 1
    else:
      print(i+1, '. ', x.name[0], ': ', [x.tag[0]])

In [ ]:
content_based_filtering(0)
collaborative_based_filtering(0)

Daftar game favorit pengguna id:  0
1 .  Insurgency :  ['Multi-player;Co-op;Cross-Platform Multiplayer;Steam Achievements;Full controller support;Steam Trading Cards;Steam Workshop;Valve Anti-Cheat enabled;Includes level editor; Action;Indie;Strategy; FPS;Realistic;Tactical']
2 .  Crusaders of the Lost Idols :  ['Single-player;Steam Achievements;Steam Trading Cards; Casual;Free to Play;Indie; Free to Play;Clicker;Casual']
3 .  Animation Throwdown: The Quest for Cards :  ['Single-player;Online Multi-Player;Cross-Platform Multiplayer;Steam Achievements;Steam Trading Cards;In-App Purchases; Adventure;Free to Play;RPG;Strategy; Free to Play;Card Game;Strategy']
4 .  Alien: Isolation :  ['Single-player;Steam Achievements;Full controller support;Steam Cloud; Action; Horror;Survival Horror;Sci-fi']
5 .  Warhammer: End Times - Vermintide :  ['Single-player;Co-op;Steam Achievements;Steam Trading Cards;Captions available;Partial Controller Support;Steam Cloud; Action;Indie; Action;Online Co-Op;C